In [1]:
import pickle
import pandas as pd
import torch

import python.voxels as voxels
import python.RNN as RNN
import python.clustering as cl

project_folder = "veleval"

In [2]:
with open("./files/"+project_folder+"/data_processed/osmnx_pathfinding_simplified.df",'rb') as infile:
    df_pathfinding = pickle.load(infile)
with open("./files/"+project_folder+"/clustering/dbscan_observations.tab",'rb') as infile:
    tab_clusters = pickle.load(infile)
with open("./files/"+project_folder+"/clustering/voxels_clustered_osmnx.dict",'rb') as infile:
    dict_voxels = pickle.load(infile)
with open("./files/"+project_folder+"/neural_networks/saved/network.param",'rb') as infile:
    param = pickle.load(infile)
with open("./files/"+project_folder+"/neural_networks/saved/num_test.tab",'rb') as infile:
    tab_num_test = pickle.load(infile)
    

size_data = 1

df = df_pathfinding

tab_routes_voxels, _, _ = voxels.generate_voxels(df, df.iloc[0]["route_num"], df.iloc[-1]["route_num"])
tab_routes_voxels_int = []

df_voxels = pd.DataFrame()
df_voxels_train = pd.DataFrame()

dict_clusters = cl.tab_clusters_to_dict(tab_clusters)

network = RNN.RNN_LSTM(size_data, max(tab_clusters)+1, param.hidden_size, param.num_layers, param.bidirectional, param.dropout)
network.load_state_dict(torch.load("files/"+project_folder+"/neural_networks/saved/network_temp.pt"))
network.eval()


FileNotFoundError: [Errno 2] No such file or directory: './files/veleval/neural_networks/saved/num_test.tab'

In [ ]:
print(len(tab_num_test))

In [ ]:
tab_results = []
tab_routes_voxels_int = []
for i in range(len(tab_routes_voxels)):
    nb_vox = 0
    tab_routes_voxels_int.append([])
    route = tab_routes_voxels[i]
    for vox in route:
        if(nb_vox%param.voxels_frequency==0): #(len(tab_routes_voxels_int[i])==0 or tab_routes_voxels_int[i][-1][0] != dict_voxels[vox]["cluster"]): 
            points = [dict_voxels[vox]["cluster"]+1]
            tab_routes_voxels_int[i].append(points)
        nb_vox += 1

In [ ]:
print(tab_routes_voxels_int[0])

In [ ]:
nb_prediction_test = 0
nb_good_prediction_test = 0
nb_prediction_train = 0
nb_good_prediction_train = 0
tab_fail_test = []
tab_fail_train = []

for i in range(len(tab_routes_voxels_int)):

    tens_route = torch.Tensor(tab_routes_voxels_int[i]).unsqueeze(1)

    hidden = network.initHidden()
    for j in range(tens_route.shape[0]):
        input = tens_route[j].unsqueeze(1)
        output, hidden = network(input, hidden)
    pred = output.argmax(dim=1, keepdim=True)
    
    tab_results.append({"probabilities": output, "good_result": pred==tab_clusters[i]})
    
    if(i in tab_num_test):
        tab_results[-1]["test"] = True
        nb_prediction_test+=1
        if(pred==tab_clusters[i]):
            nb_good_prediction_test+=1
    else:
        tab_results[-1]["test"] = False
        nb_prediction_train+=1
        if(pred==tab_clusters[i]):
            nb_good_prediction_train+=1
    
            
print("Good train predict:", nb_good_prediction_train/nb_prediction_train*100, "%")
print("Good test predict:", nb_good_prediction_test/nb_prediction_test*100, "%")

In [ ]:
print(len(tab_routes_voxels_int), len(tab_routes_voxels))